# Step 2. Authenticate to IBM Cloud with APIKEY

Use api key you copied from earlier and enter it in below. 

You will need to specify the region based on the location of your Power VS Service instance. Use this URL to list the proper region: https://cloud.ibm.com/docs/power-iaas?topic=power-iaas-creating- power-virtual-server

In [ ]:
# import utils as u
# Fill in APIKEY value below
!ibmcloud config --check-version=false
APIKEY='INSERT IBM CLOUD API KEY HERE'
IC_REGION='INSERT IBM COULD POWERVS REGION HERE'
!ibmcloud login -a https://cloud.ibm.com --apikey $APIKEY -r $IC_REGION

You should now be logged into the ibmcloud cli. Your APIKEY has automatically logged you into the correct account.

## Step 2a. Collect your IBM PowerVS Service CRN

Next we will use the IBM Cloud CLI to gather some meta data about our PowerVS Instance so that we can provision a server with the ibmcloud CLI. 

In particular we will need to get the PowerVS instance crn so that we can create new LPARS inside of it.
Run the command in the next cell and copy your instance crn

In [ ]:
# Run ibmcloud pi to get instance id 
# note pi stands for power-iaas 
!ibmcloud pi service-list

The instance_crn is a substring in the long output from ibmcloud pi service-list. The CRN will be used to target your PowerVS instance so you can run commands against it

Use the example screenshot to grab your corresponding instance_crn and assign to the variable below.

![](Images/instance_crn_screenshot.png)

## Step 2b. Target your PowerVS Service

Here we will target our PowerVS Instance so that future IBM Cloud CLI commands are directed to your instance.

In [ ]:
#Get instance crn 
INSTANCE_CRN='PASTE YOUR SERVICE CRN HERE'

#! ibmcloud pi service-target crn:v1:bluemix:public:xxxxxxxxxxxxxx
!ibmcloud pi service-target $INSTANCE_CRN

# See output from other IBM CLI commands

IBM Cloud CLI is a full featured tool to administer all parts of IBM cloud. 

We make use of the power-iaas module for commands to help us administer PowerVS

For more documentation, see this link https://cloud.ibm.com/docs/power-iaas-cli-plugin?topic=power-iaas-cli-plugin-power-iaas-cli-reference

Feel free to change the cli command in the code block below to try and see different results

In [ ]:
#List all PowerVS instances (LPARs) inside your targeted PowerVS service
!ibmcloud pi ins

## Samples of other commands ...
# list images
# ibmcloud pi imglc -i
# # list networks
# ibmcloud pi nets
# # Show LPARS that are deployed
# ibmcloud pi ins
# # show LPAR details
# ibmcloud pi in 042434e5-e049-4e20-b712-80def7775dc1
# # show storage volumes in PVS
# ibmcloud pi vols

# Create an LPAR using IBM Cloud CLI

Here we will show you how to create an LPAR using the IBM Cloud CLI. 

To do this we: 
* create a private network
* create an LPAR on that private network

There are more complex configurations you can use to add storage volumes and
configure VM. See documentation for more advanced examples.

In [ ]:
# Create Private network
# NOTE : CREATE A UNIQUE NAME For your network, and server in following example
import numpy as np

UNIQUE_NETWORK_NAME="ibmcloud-sample1_" + str(np.random.randint(0,1000)) 
print(UNIQUE_NETWORK_NAME)

!ibmcloud pi network-create-private $UNIQUE_NETWORK_NAME --cidr-block 10.0.10.0/24 --ip-range "10.0.10.2-10.0.10.200"



In [ ]:
# List out available images to deploy
!ibmcloud pi imglc

In [ ]:
# Pick and image from above and deploy !
UNIQUE_SERVER_NAME="ibmcloud-sample1_" + str(np.random.randint(0,1000)) 
IMAGE_ID="PASTE YOUR IMAGE ID HERE"
!ibmcloud pi inc $UNIQUE_SERVER_NAME --network $UNIQUE_NETWORK_NAME --image $IMAGE_ID --memory 2 --processors 0.25 --processor-type shared --storage-type tier3

# Create/Reuse PowerVS SSH Key

If you completed the steps from lab 1, you will have already created an SSH key that we can use. If you didnt finish that part, no problem, the following screen shots show you how to create an SSH login key for your PowerVS LPARs you will create.

An alternative option to getting your public key is to use the IBM Cloud CLI as shown below. As optional exercise, try to create a key using IBM cloud CLI tool instead of GUI.

In [ ]:
# If you want to avoid the GUI, you can use the ibmcloud tool to grab your key
!ibmcloud pi keys

## Instructions to create ssh key via GUI

First go back to your dashboard view and click 'Services & Software'
![](Images/dashboard.png)


Next click the link for your PowerVS Instance you created. In the example screenshot we named it MattR_PVS
![](Images/services_list.png)


Finally on the left sidebar click 'SSH keys'. If you don't have a key here, then just go ahead and create a new key with a meaningful name. Record the key value, and enter the data in the python cell below ...
![](Images/ssh_keys.png)

In [ ]:
# Enter SSH key discovered from above steps # must start with ssh-rsa
SSH_KEY="PASTE YOUR SSH KEY HERE"

# Example of Snapshots using Command Line Interface (CLI)

This will allow you to save the entire image of your Power Virtual Server environment to allow you to duplicate or re-create a virtual machine with the settings and software installed on the PowerVS instance.

Snapshot has been added to the ibmcloud pi command line, full examples require authentication (either via "login" or with the enviromental API Key) and a "service-target" selection

In [ ]:
## Start by Listing the VMs (instances)
!ibmcloud pi ins

In [ ]:
## Create a snapshot
INSTANCE_ID='PASTE YOUR INSTANCE ID HERE'
!ibmcloud pi snapshot-create $INSTANCE_ID --name snaptest2 --description "My test snapshot"

A snapshot id will be generated for you to reference the snapshot.

Next we will ist all of the snapshots to validate if it has been created and is available

In [ ]:
## List Snapshots
!ibmcloud pi snapshots

# Example of Cloning using Command Line Interface (CLI)

Cloning is a command which is part of the ibmcloud pi command line. 

Full examples require authentiation (either via "login" or with an environmental API KEY) and a "service-target" selection.

In [ ]:
# First lets list all the disks available
!ibmcloud pi volumes

Find the virtual machine (VM) you would like to clone and copy the id of it

In [ ]:
VMID='PASTE VOLUME ID HERE'
!ibmcloud pi volume-create-clone MDBCOPY --volumes $VMID

Lastly, you may check the status of your cloning. You will need to grab the task ID given from the output above.

In [ ]:
TASKID='PASTE TASK ID HERE' 
!ibmcloud pi volume-clone $TASKID

# Step 7: Working with the IBM pvsadm tool

This will allow you to make changes and/or query your Power Virtual Server environment to allow you to administer it from an IT Admin perspective.

This is a tool to help with managing of resources in IBM Power Systems Virtual Server:
https://www.ibm.com/cloud/power-virtual-server

Although this is already installed for you in this notebook you can find more details on the management and installation of this command here: https://github.com/ppc64le-cloud/pvsadm

In [ ]:
## View the basic options
!pvsadm -h

## The purge subcommand will remove a lot! Use with caution! 
!pvsadm purge -h

**Warning:** The pvsadm purge process removes ALL VMs in a PowerVS Service Instance, so be careful using that!

Users of PowerVS often would like additional details from the small pop-up notification windows that temporarily appear during operations. The first exercise uses the get events task for pvsadm.

In [ ]:
## Start by Listing the VMs (instances)
!ibmcloud pi sl

In [ ]:
# Select name of service you want to see events for. Make sure it is the name and not the crn
PVS_NAME="ENTER POWERVS NAME HERE (NOT CRN)"

# !pvsadm get events -k $APIKEY -i $PVS_NAME
!pvsadm get events -k $APIKEY -n $PVS_NAME

In this scenario we will give you an image file and you will leverage pvsadm to upload it to Cloud Object Storage.

In [ ]:
!pvsadm image upload --help

In [ ]:
!pvsadm image upload -k $APIKEY -b "ENTER YOUR IBM COS BUCKET NAME" -f /tmp/alpine-ppc64le.iso -r "ENTER YOUR IBM COS BUCKET REGION"

# Step 8. Edit Ansible Playbooks

Ansible playbooks are files that contain the instructions to perform a set of ansible tasks. Here we downloaded 2 samples from the PowerVS sample reposistory in github : https://github.com/IBM-Cloud/ansible-collection-ibm/tree/master/examples/simple-vm-power-vs

We pre-loaded two of these samples for your reference. 
* list_pi_images.yml
* create.yml

You will need to edit these files based on the screenshots below prior to running them. Jupyter lab comes with a built-in code editor to make this easier.

## Step 8a. Open file list_pi_images.yml

This playbook will list available boot images for you to use with your create LPAR playbook in a subsequent step.

On the left sidebar click list_pi_images.yml and edit this file with your PowerVS instance_id. See below commandfor collecting the instance id.

In [ ]:
# find your instance id
!ibmcloud pi service-list

Get your instance id. This is the last section of your crn

![](Images/Instance_id_screenshot.png)

Take your instance id value and edit list_pi_images.yml with proper value

![](Images/list_pi_images_screenshot.png)

# Step 9. Run list_pi_images.yml playbook

For this playbook to run we will need to set three environment variables. These variabels can also be included as variables in the playbook but in this instance we will use environment variables.

* IC_API_KEY: Your IBM API Key created in step 1. Used for Authenication
* IC_REGION: Used to address the correct api endpoint. See Endpoint URL section of [this link](https://cloud.ibm.com/apidocs/power-cloud).
* IC_ZONE: Used to address the correct DC in relevant region. See IBM Power Infrastructure Zone on [this link](https://cloud.ibm.com/docs/power-iaas?topic=power-iaas-creating-power-virtual-server).

In [ ]:
#Save the instance id for the next playbook
INSTANCE_ID="PASTE YOUR INSTANCE ID HERE"

# Running ansible playbooks requires IC_API_KEY environment variable (and other epending on playbook) is set 
# 2. run ansible playbook using bash shell

!export IC_API_KEY=$APIKEY IC_REGION='POWERVS REGION' IC_ZONE='POWERVS DC'; \
ansible-playbook list_pi_images.yml

Here, the output shows you the available boot images that you can use to deploy an LPAR. If this list is blank, this means you need to import one of the base images into your boot image list.**

Expected output could look something like this ...
```
        ok: [localhost] => {
           "images_list.resource.image_info | items2dict(key_name='name', value_name='id')": {
              "7200-05-01": "860f619a-0061-404d-a465-0167b5dced8b",
              "IBMi-74-01-001": "50102bf2-74a1-4544-b87e-43932c7459bd"
               }
        }
```

**Make note of available image to use in our next step to create an LPAR**

# Step 10 :Modify create.yml : Create AIX LPAR

Finally open and edit the create.yml file based on the following screenshot. When complete with edits save the file.

![](Images/create_screenshot.png)

In [ ]:
# Use these values to edit your file .. #print("****")
#print("SSH_KEY")
print(SSH_KEY)
print("****") 
#print("INSTANCE_ID") 
print(INSTANCE_ID)

In [ ]:
#This playbook will create the LPAR/VM
!export IC_API_KEY=$APIKEY IC_REGION='POWERVS REGION' IC_ZONE='POWERVS DC'; \
ansible-playbook create.yml

# Step 11. Verify LPAR creation in IBM Cloud UI

![](Images/ansible_result.png)

# Lab Complete

Congratulations, you have now learned about tools that you can use to provision services within the IBM PowerVS Cloud.